In [1]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# set path
path = "C:/Users/smvan/CFFS-S23/CFFS-22-23"
os.chdir(path)
print(path)

C:/Users/smvan/CFFS-S23/CFFS-22-23


In [3]:
#OK_menu = pd.read_csv('data/final/2022_2023_CFFS_Outcomes/Data_Labelled_Gather22-23.csv', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# FOR OK
# OK_menu = pd.read_csv('data/data_for_calculating_baseline/OK_19-20_Recipes_GHG.csv', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# FOR GATHER
OK_menu = pd.read_csv('data/data_for_calculating_baseline/Gather_2019_recipes.csv', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
# OK_menu

# FOR TOTEM
# OK_menu = pd.read_csv('data/data_for_calculating_baseline/Totem_2019_recipes.csv', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

OK_menu

,ProdId,Description,SalesGroup,GHG Emission (g),Weight (g),GHG Emission (Kg),GHG Emission (Kg) / 100g,GHG Emission (g) / 100g,Sales,GHG Emission (g) / 100g * sales
0,R-53080,SL|Lamb Roast|ala carte,GV SIMPLY LOCAL,9119.828773,120.000,9.1198,7.599857,7599.857311,61,4.635913e+05
1,R-44681,SAND|Add Roast Beef,GV-SANDWICH,3184.784299,50.000,3.1848,6.369569,6369.568598,271,1.726153e+06
2,R-53405,SL|Beef Back Rib,GV SIMPLY LOCAL,12396.520080,284.000,12.3965,4.364972,4364.971858,12,5.237966e+04
3,R-49123,SL|Beef Brisket|115g,GV SIMPLY LOCAL,6399.166309,165.000,6.3992,3.878283,3878.282612,1,3.878283e+03
4,R-54321,DIM SUM|Beef Stir Stirfry|SP,GV - DIM SUM,9580.520376,384.000,9.5805,2.494927,2494.927181,123,3.068760e+05
...,...,...,...,...,...,...,...,...,...,...
264,R-14102,SIDE|SoupCongee|Plain,GV - SOUP,146.234337,453.592,0.1462,0.032239,32.239179,416,1.341150e+04
265,R-13177,G&G|Spread|Jam & Peanut Butter,GV - GRAB & GO,0.570205,3.173,0.0006,0.017971,17.970526,5041,9.058942e+04
266,R-50508,GZA|Soup|Miso,GV GYAZA,75.297701,485.000,0.0753,0.015525,15.525299,1,1.552530e+01
267,R-49912,GZA|Miso for cone|SP,SPECIALS,1.129129,185.000,0.0011,0.000610,0.610340,101,6.164434e+01


In [4]:
# OK_all_factors = pd.read_csv('data/data_for_calculating_baseline/OK 21-22 Recipes Footprints.csv')
# Products_List_2019.csv
# OK_all_factors = pd.read_csv('data/data_for_calculating_baseline/Products_List_2019.csv')
# OK_all_factors = pd.read_csv('data/preprocessed/Products_List.csv')
#OK_all_factors = pd.read_csv('data/final/2022_2023_CFFS_Outcomes/Data_Labelled_Gather22-23.csv')
OK_all_factors = pd.read_csv('data/final/Recipes Footprints.csv')
OK_all_factors[OK_all_factors['ProdId'] == 'R-23938']
# OK_all_factors

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),km^2 land use/kg product,Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,m^2 land use/kg product / 100g


In [5]:
OK_menu = OK_menu[OK_menu['Weight (g)'] >= 6]

In [6]:
OK_menu

,ProdId,Description,SalesGroup,GHG Emission (g),Weight (g),GHG Emission (Kg),GHG Emission (Kg) / 100g,GHG Emission (g) / 100g,Sales,GHG Emission (g) / 100g * sales
0,R-53080,SL|Lamb Roast|ala carte,GV SIMPLY LOCAL,9119.828773,120.000,9.1198,7.599857,7599.857311,61,4.635913e+05
1,R-44681,SAND|Add Roast Beef,GV-SANDWICH,3184.784299,50.000,3.1848,6.369569,6369.568598,271,1.726153e+06
2,R-53405,SL|Beef Back Rib,GV SIMPLY LOCAL,12396.520080,284.000,12.3965,4.364972,4364.971858,12,5.237966e+04
3,R-49123,SL|Beef Brisket|115g,GV SIMPLY LOCAL,6399.166309,165.000,6.3992,3.878283,3878.282612,1,3.878283e+03
4,R-54321,DIM SUM|Beef Stir Stirfry|SP,GV - DIM SUM,9580.520376,384.000,9.5805,2.494927,2494.927181,123,3.068760e+05
...,...,...,...,...,...,...,...,...,...,...
263,R-53356,POP|Fennel Bouillabaisse|MR,SPECIALS,83.284362,236.000,0.0833,0.035290,35.289984,62,2.187979e+03
264,R-14102,SIDE|SoupCongee|Plain,GV - SOUP,146.234337,453.592,0.1462,0.032239,32.239179,416,1.341150e+04
266,R-50508,GZA|Soup|Miso,GV GYAZA,75.297701,485.000,0.0753,0.015525,15.525299,1,1.552530e+01
267,R-49912,GZA|Miso for cone|SP,SPECIALS,1.129129,185.000,0.0011,0.000610,0.610340,101,6.164434e+01


In [21]:
count = 0
for i in OK_all_factors['ProdId']:
    for j in OK_menu['ProdId']:
        if (i == j):
            row = OK_menu[OK_menu['ProdId'] == i].index[0]
            row2 = OK_all_factors[OK_all_factors['ProdId'] == i].index[0]
            OK_menu.loc[row, 'N lost (g) / 100g'] = OK_all_factors.loc[row2, 'N lost (g) / 100g']
            OK_menu.loc[row, 'Stress-Weighted Water Use (L) / 100g'] = OK_all_factors.loc[row2, 'Stress-Weighted Water Use (L) / 100g']
            OK_menu.loc[row, 'm^2 land use/kg product / 100g'] = OK_all_factors.loc[row2, 'm^2 land use/kg product / 100g']
            OK_menu.loc[row, '100grams'] = 2

# for i in range(OK_menu.shape[0]):
#     OK_menu.loc[i, '100grams'] = OK_menu.loc[i, '100grams'].value * 100

OK_menu.rename(columns={'100grams': '1000000grams'}, inplace = True)
OK_menu['1000000grams'] *= 1000000
        
#OK_menu = OK_menu.drop('100g', axis = 1)
            
#print(count)
#OK_all_factors[OK_all_factors['Description'] == 'SAND|Add|Beef'].index[0]

# OK_menu.loc[:, 'N lost (g) / 100g'] = 0
# OK_menu['Stress-Weighted Water Use (ml) / 100g'] = 0
OK_menu

# for row index 2 N lost: 0.75 of OK_menu
# for row index 2 Stress lost: 384838.80 of OK_menu
#OK_menu.loc[2, 'Stress-Weighted Water Use (ml) / 100g']
#OK_menu.loc[2, 'N lost (g) / 100g']
# OK_all_factors[OK_all_factors['Description'] == 'VEG|Southwest Bowl']
# Above are checks to see if the N lost and stress water data match

,ProdId,Description,SalesGroup,GHG Emission (g),Weight (g),GHG Emission (Kg),GHG Emission (Kg) / 100g,GHG Emission (g) / 100g,Sales,GHG Emission (g) / 100g * sales,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g,m^2 land use/kg product / 100g,1000000grams
0,R-53080,SL|Lamb Roast|ala carte,GV SIMPLY LOCAL,9119.828773,120.000,9.1198,7.599857,7599.857311,61,4.635913e+05,42.24,86.48,26136.16,2000000.0
1,R-44681,SAND|Add Roast Beef,GV-SANDWICH,3184.784299,50.000,3.1848,6.369569,6369.568598,271,1.726153e+06,50.75,9446.24,19481.34,2000000.0
2,R-53405,SL|Beef Back Rib,GV SIMPLY LOCAL,12396.520080,284.000,12.3965,4.364972,4364.971858,12,5.237966e+04,34.73,6471.15,13349.82,2000000.0
3,R-49123,SL|Beef Brisket|115g,GV SIMPLY LOCAL,6399.166309,165.000,6.3992,3.878283,3878.282612,1,3.878283e+03,30.70,5732.27,11865.69,2000000.0
4,R-54321,DIM SUM|Beef Stir Stirfry|SP,GV - DIM SUM,9580.520376,384.000,9.5805,2.494927,2494.927181,123,3.068760e+05,19.85,3784.78,7498.46,2000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,R-53356,POP|Fennel Bouillabaisse|MR,SPECIALS,83.284362,236.000,0.0833,0.035290,35.289984,62,2.187979e+03,0.44,129.11,54.64,2000000.0
264,R-14102,SIDE|SoupCongee|Plain,GV - SOUP,146.234337,453.592,0.1462,0.032239,32.239179,416,1.341150e+04,0.08,51.36,42.27,2000000.0
266,R-50508,GZA|Soup|Miso,GV GYAZA,75.297701,485.000,0.0753,0.015525,15.525299,1,1.552530e+01,0.06,1.59,36.49,2000000.0
267,R-49912,GZA|Miso for cone|SP,SPECIALS,1.129129,185.000,0.0011,0.000610,0.610340,101,6.164434e+01,0.01,1.41,1.30,2000000.0


In [8]:
# Results for Totem
# OK_menu.to_csv('data/data_for_calculating_baseline/Totem_results/totem_results.csv', index = False)

# Results for OK
# OK_menu.to_csv('data/data_for_calculating_baseline/OK_results/ok_results.csv', index = False)

# Results for Gather
OK_menu.to_csv('data/data_for_calculating_baseline/Gather_results/gather_results.csv', index = False)